In [ ]:
import mariadb
pool = None
user = "root"
password = "mariadb"
host = "mariadb.host"
port = 3306
database = "istock"

def initConnectionPool():
    global pool
    pool = mariadb.ConnectionPool(
        user=user,
        password=password,
        host=host,
        port=port,
        database=database,
        pool_name="app",
        pool_size=8
    )
    return pool


def getConnection():
    try:
        if pool is None:
            initConnectionPool()
        conn = pool.get_connection()
    except mariadb.Error as e:
        print(f'Error connecting to MariaDB Platform: {e}')
    return conn

initConnectionPool()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_sql('select * from stock_kline t where t.code = "0.002027"', con=getConnection())

In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [ ]:
df = df.set_index('date')

In [ ]:
last = df['2020':].tail(9)

In [ ]:
last

In [ ]:
"""
n日RSV=（Cn－Ln）/（Hn－Ln）*100
公式中，Cn为第n日收盘价；Ln为n日内的最低价；Hn为n日内的最高价。
当日K值=2/3*前一日K值+1/3*当日RSV
当日D值=2/3*前一日D值+1/3*当日K值
J值=3*当日K值-2*当日D值
一般n取9
"""
cn = last['close'].tail(1)[0]
ln = last['low'].min()
hn = last['high'].max()
rsv = (cn - ln) / (hn - ln) * 100
print('cn={},ln={},hn={},rsv={}'.format(cn, ln, hn, rsv))

In [ ]:
kn = (1 - 1 / 3) * 50 + 1 / 3 * rsv
dn = (1 - 1 / 3) * 50 + 1 / 3 * kn
j = 3 * kn + 2 * dn
print('k={},d={},j={}'.format(kn, dn, j))